In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import re
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func

file_path1 = "Resources/artists.csv"
file_path2 = "Resources/tracks.csv"

In [2]:
artists_df = pd.DataFrame(pd.read_csv(file_path1))
artists_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162095 entries, 0 to 1162094
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   id          1162095 non-null  object 
 1   followers   1162084 non-null  float64
 2   genres      1162095 non-null  object 
 3   name        1162095 non-null  object 
 4   popularity  1162095 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 44.3+ MB


In [3]:
artists_df.head()

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0


In [4]:
mask = artists_df['genres'] != '[]'
artists_df1 = artists_df.loc[mask]
artists_df2 = artists_df1.reset_index(drop=True)
artists_df3 = artists_df2.dropna(axis=0)
artists_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305590 entries, 0 to 305594
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          305590 non-null  object 
 1   followers   305590 non-null  float64
 2   genres      305590 non-null  object 
 3   name        305590 non-null  object 
 4   popularity  305590 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 14.0+ MB


In [5]:
tracks_df = pd.DataFrame(pd.read_csv(file_path2))
tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586672 entries, 0 to 586671
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                586672 non-null  object 
 1   name              586601 non-null  object 
 2   popularity        586672 non-null  int64  
 3   duration_ms       586672 non-null  int64  
 4   explicit          586672 non-null  int64  
 5   artists           586672 non-null  object 
 6   id_artists        586672 non-null  object 
 7   release_date      586672 non-null  object 
 8   danceability      586672 non-null  float64
 9   energy            586672 non-null  float64
 10  key               586672 non-null  int64  
 11  loudness          586672 non-null  float64
 12  mode              586672 non-null  int64  
 13  speechiness       586672 non-null  float64
 14  acousticness      586672 non-null  float64
 15  instrumentalness  586672 non-null  float64
 16  liveness          58

In [6]:
tracks_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [7]:
#Parse release_date into date time
release_date = tracks_df["release_date"]
release_date = pd.to_datetime(release_date, format='%Y-%m-%d', exact=True)
tracks_df['release_date'] = release_date
tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586672 entries, 0 to 586671
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                586672 non-null  object        
 1   name              586601 non-null  object        
 2   popularity        586672 non-null  int64         
 3   duration_ms       586672 non-null  int64         
 4   explicit          586672 non-null  int64         
 5   artists           586672 non-null  object        
 6   id_artists        586672 non-null  object        
 7   release_date      586672 non-null  datetime64[ns]
 8   danceability      586672 non-null  float64       
 9   energy            586672 non-null  float64       
 10  key               586672 non-null  int64         
 11  loudness          586672 non-null  float64       
 12  mode              586672 non-null  int64         
 13  speechiness       586672 non-null  float64       
 14  acou

In [8]:
#Use a mask to find tracks that are more recent than January 1, 1960
mask = tracks_df['release_date'] > '1960-01-01'
tracks_df1 = tracks_df.loc[mask]

#drop 'time signature' and 'id_artists' columns
tracks_df2 = tracks_df1.drop(columns = ['time_signature', 'id_artists'])

#drop faulty row
tracks_df3 = tracks_df2.drop([509271])

#reset and drop index
tracks_df4 = tracks_df3.reset_index(drop=True)

#drop empty rows
tracks_df5 = tracks_df4.dropna(axis=0)

tracks_df5.head()

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2G0GextMwZJLkNxcSZ7ZJ3,(What A) Wonderful World - Mono,67,128787,0,['Sam Cooke'],1960-02-01,0.686,0.672,11,-5.523,1,0.0323,0.700,0.000000,0.1350,0.857,128.550
1,3oAWTk92mZBxKBOKf8mR5v,Summertime Blues,64,119360,0,['Eddie Cochran'],1960-05-01,0.714,0.886,11,-8.629,0,0.0554,0.116,0.184000,0.1800,0.954,156.351
2,2x6pbpjVGjiWCcH89IK8AX,Breaking Up Is Hard to Do,63,139200,0,['Neil Sedaka'],1960-12-30,0.743,0.799,8,-5.466,0,0.0375,0.699,0.000000,0.0635,0.965,116.112
3,47mA6f44zxLtdATOoY7GjN,Georgia on My Mind - Original Master Recording,61,217415,0,['Ray Charles'],1960-09-01,0.138,0.399,7,-8.756,1,0.0311,0.782,0.000004,0.1880,0.296,179.930
4,0DICNd5XQ1og9UeYzxoNFV,Baby (You've Got What It Takes),60,165760,0,"['Dinah Washington', 'Brook Benton']",1960-07-05,0.670,0.596,3,-9.347,1,0.0627,0.852,0.002030,0.6530,0.813,133.396


In [11]:
#Output csv's for PostgreSQL
artists_df3.to_csv('CSV/artists_df_pipes.csv', index=False, sep='|')
tracks_df5.to_csv('CSV/tracks_df_pipes.csv', index=False, sep='|')

In [12]:
#Output for regular CSV's
artists_df3.to_csv('CSV/artists_df.csv', index=False)
tracks_df5.to_csv('CSV/tracks_df.csv', index=False)